# Data Exploration
- This notebook performs exploratory data analysis on the dataset.
- To expand on the analysis, attach this notebook to a cluster with runtime version **16.3.x-cpu-ml-scala2.12**,
edit [the options of pandas-profiling](https://pandas-profiling.ydata.ai/docs/master/rtd/pages/advanced_usage.html), and rerun it.
- Explore completed trials in the [MLflow experiment](#mlflow/experiments/4297320214106197).

In [0]:
%pip install --no-deps ydata-profiling==4.8.3 pandas==2.2.3 visions==0.7.6 tzdata==2024.2

  Using cached ydata_profiling-4.8.3-py2.py3-none-any.whl.metadata (20 kB)


  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached visions-0.7.6-py3-none-any.whl.metadata (11 kB)


  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached ydata_profiling-4.8.3-py2.py3-none-any.whl (359 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)


Using cached visions-0.7.6-py3-none-any.whl (104 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)


  Attempting uninstall: ydata-profiling
    Found existing installation: ydata-profiling 4.9.0
    Not uninstalling ydata-profiling at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-daa344d1-3273-48e8-8b47-ede6c7ed87ec
    Can't uninstall 'ydata-profiling'. No files were found to uninstall.


  Attempting uninstall: visions
    Found existing installation: visions 0.7.5
    Not uninstalling visions at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-daa344d1-3273-48e8-8b47-ede6c7ed87ec
    Can't uninstall 'visions'. No files were found to uninstall.


  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Not uninstalling pandas at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-daa344d1-3273-48e8-8b47-ede6c7ed87ec
    Can't uninstall 'pandas'. No files were found to uninstall.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [0]:
import mlflow
import os
import uuid
import shutil
import pandas as pd
import databricks.automl_runtime

# Download input data from mlflow into a pandas DataFrame
# Create temporary directory to download data
temp_dir = os.path.join(os.environ["SPARK_LOCAL_DIRS"], "tmp", str(uuid.uuid4())[:8])
os.makedirs(temp_dir)

# Download the artifact and read it
training_data_path = mlflow.artifacts.download_artifacts(run_id="e342157b8ede49668a32741350c5c4bb", artifact_path="data", dst_path=temp_dir)
df = pd.read_parquet(os.path.join(training_data_path, "training_data"))

# Delete the temporary data
shutil.rmtree(temp_dir)

target_col = "Personal_Loan"

# Drop columns created by AutoML and user-specified sample weight column (if applicable) before pandas-profiling
df = df.drop(['_automl_split_col_0000'], axis=1)

# Convert columns detected to be of semantic type numeric
numeric_columns = ["CCAvg", "Mortgage"]
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors="coerce")

Thu Aug  7 05:05:06 2025 Connection to spark from PID  90438
Thu Aug  7 05:05:06 2025 Initialized gateway on port 42599


Thu Aug  7 05:05:07 2025 Connected to spark.


## Semantic Type Detection Alerts

For details about the definition of the semantic types and how to override the detection, see
[Databricks documentation on semantic type detection](https://docs.databricks.com/applications/machine-learning/automl.html#semantic-type-detection).

- Semantic type `numeric` detected for columns `CCAvg`, `Mortgage`. Training notebooks will convert each column to a numeric type and encode features based on numerical transformations.

## Profiling Results

In [0]:
from ydata_profiling import ProfileReport
df_profile = ProfileReport(df,
                           correlations={
                               "auto": {"calculate": True},
                               "pearson": {"calculate": True},
                               "spearman": {"calculate": True},
                               "kendall": {"calculate": True},
                               "phi_k": {"calculate": True},
                               "cramers": {"calculate": True},
                           }, title="Profiling Report", progress_bar=False, infer_dtypes=False)
profile_html = df_profile.to_html()

displayHTML(profile_html)

/local_disk0/.ephemeral_nfs/envs/pythonEnv-daa344d1-3273-48e8-8b47-ede6c7ed87ec/lib/python3.12/site-packages/ydata_profiling/model/pandas/correlations_pandas.py:36: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  return df.corr(method="pearson")
/local_disk0/.ephemeral_nfs/envs/pythonEnv-daa344d1-3273-48e8-8b47-ede6c7ed87ec/lib/python3.12/site-packages/ydata_profiling/model/pandas/correlations_pandas.py:29: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  return df.corr(method="spearman")
/local_disk0/.ephemeral_nfs/envs/pythonEnv-daa344d1-3273-48e8-8b47-ede6c7ed87ec/lib/python3.12/site-packages/ydata_profiling/model/pandas/correlations_pandas.py:43: 

Number of variables,12
Number of observations,5000
Missing cells,0
Missing cells (%),0.0%
Duplicate rows,13
Duplicate rows (%),0.3%
Total size in memory,468.9 KiB
Average record size in memory,96.0 B
Text,10
Numeric,2
Dataset has 13 (0.3%) duplicate rows,Duplicates
